In [1]:
base_dir = 'datasets/cats_and_dogs'

In [2]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [3]:
from keras.layers import *
from keras.models import *

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [4]:
print('weight count before conv_base freezing: ', len(model.trainable_weights))
conv_base.trainable = False
print('weight count after conv_base freezing: ', len(model.trainable_weights))
model.summary()

weight count before conv_base freezing:  30
weight count after conv_base freezing:  4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
import os
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(150, 150),
    batch_size = 20,
    class_mode = 'binary'
)

val_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'validation'),
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

In [8]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=val_generator,
    validation_steps=50
).history

Epoch 1/30
100/100 [==============================] - 17s 171ms/step - loss: 0.5992 - acc: 0.6775 - val_loss: 0.4755 - val_acc: 0.7980
Epoch 2/30
100/100 [==============================] - 15s 150ms/step - loss: 0.4918 - acc: 0.7805 - val_loss: 0.4041 - val_acc: 0.8220
Epoch 3/30
100/100 [==============================] - 16s 158ms/step - loss: 0.4419 - acc: 0.7995 - val_loss: 0.3396 - val_acc: 0.8600
Epoch 4/30
100/100 [==============================] - 15s 149ms/step - loss: 0.4099 - acc: 0.8180 - val_loss: 0.3141 - val_acc: 0.8660
Epoch 5/30
100/100 [==============================] - 15s 149ms/step - loss: 0.3782 - acc: 0.8420 - val_loss: 0.2981 - val_acc: 0.8720
Epoch 6/30
100/100 [==============================] - 15s 148ms/step - loss: 0.3915 - acc: 0.8180 - val_loss: 0.2901 - val_acc: 0.8690
Epoch 7/30
100/100 [==============================] - 16s 157ms/step - loss: 0.3698 - acc: 0.8380 - val_loss: 0.2806 - val_acc: 0.8780
Epoch 8/30
100/100 [==============================] - 1

In [9]:
import matplotlib.pyplot as plt
epoch = range(1, len(history['acc']) + 1)
plt.clf()
plt.plot(epoch, history['acc'], 'bo', label='Training Acc')
plt.plot(epoch, history['loss'], 'bP', label='Training Loss')
plt.plot(epoch, history['val_acc'], 'ro', label='Validation Acc')
plt.plot(epoch, history['val_loss'], 'rP', label='Validation Loss')
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
score = model.evaluate_generator(test_generator, 50)

In [11]:
print("Loss : ", score[0], "\tACC : ", score[1])

Loss :  0.24786635518074035 	ACC :  0.8919999969005584
